In [1]:
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

In [2]:
f = open("../../dataset/sense/dict_sense-keys", 'rb')
dict_sense_keys = pickle.load(f)
f.close()

f = open("../../dataset/sense/dict_word-sense", 'rb')
dict_word_sense = pickle.load(f)
f.close()

f = open('../Glove/word_embedding_glove', 'rb')
word_embedding = pickle.load(f)
f.close()
word_embedding = word_embedding[: len(word_embedding)-1]

f = open('../Glove/vocab_glove', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

with open('../../dataset/raw_preprocess_train','rb') as f:
    data=pickle.load(f)

with open('../../dataset/fulldata_vocab_sense','rb') as f:
    vocab_lex=pickle.load(f)

lex2id = dict((s, i) for i,s in enumerate(vocab_lex))
id2lex = dict((i, s) for i,s in enumerate(vocab_lex))

print(len(vocab_lex))
max_sent_size = 200

46


In [3]:
_pos = []
for i in range(len(data)):
    for pp in data[i][4]:
        _pos.append(pp)
        
pos_count = Counter(_pos)
pos_count = pos_count.most_common()
vocab_pos = [pp for pp, c in pos_count]
pos2id = dict((s, i) for i,s in enumerate(vocab_pos))
print(len(vocab_pos))

12


In [4]:
data_y1 = []
data_y2 = []
data_y3 = []
for i in range(len(data)):
    if (len(data[i][1])<=200):
        for j in range(len(data[i][2])):
            if data[i][2][j] is not None:
                data_y1.append(dict_sense_keys[data[i][2][j]][3])
                data_y2.append(dict_sense_keys[data[i][2][j]][4])
                data_y3.append(dict_sense_keys[data[i][2][j]][5])

sense_count1 = Counter(data_y1)
sense_count1 = sense_count1.most_common()[:-2]

sense_count2 = Counter(data_y2)
sense_count2 = sense_count2.most_common(180)

sense_count3 = Counter(data_y3)
sense_count3 = sense_count3.most_common(300)

In [5]:
vocab_sense1 = [s for s, c in sense_count1]
vocab_sense2 = [s for s, c in sense_count2]
vocab_sense3 = [s for s, c in sense_count3]

vocab_sense1 = sorted(vocab_sense1, key=lambda x:int(x[0]))
vocab_sense2 = sorted(vocab_sense2, key=lambda x:int(x[0]))
vocab_sense3 = sorted(vocab_sense3, key=lambda x:int(x[0]))

In [6]:
new_vocab_pos = ['NOUN', 'VERB', 'ADJ', 'ADV']
for v in vocab_pos:
    if v not in new_vocab_pos:
        new_vocab_pos.append(v)
vocab_pos = new_vocab_pos

In [20]:
def make_mask_matrix(vocab_sense):
    mask_mat = []
    sense_list = [int(string[0]) if int(string[0]) !=5 else 3 for string in vocab_sense]
    for i in range(len(set(sense_list))):
        temp_row=[0]*len(sense_list)
        for j in range(len(sense_list)):
            if sense_list[j] == i+1:
                temp_row[j]= 1
            else:
                temp_row[j]= 0
        mask_mat.append(temp_row)
    return mask_mat

def hard_make_mask_matrix(vocab_sense):
    mask_mat = []
    sense_list = [int(string[0]) if int(string[0]) !=5 else 3 for string in vocab_sense]
    for i in range(len(set(sense_list))):
        temp_row=[0]*len(sense_list)
        for j in range(len(sense_list)):
            if sense_list[j] == i+1:
                temp_row[j]= 0
            else:
                temp_row[j]= -10
        mask_mat.append(temp_row)
    return mask_mat

In [62]:
mask_mat1 = make_mask_matrix(vocab_sense1)
mask_mat2 = make_mask_matrix(vocab_sense2)
mask_mat3 = make_mask_matrix(vocab_sense3)

with open('../../dataset/train_val_data_fine/mask_mat_lex','wb') as f:
    pickle.dump(mask_mat1, f)
with open('../../dataset/train_val_data_fine/mask_mat_sense','wb') as f:
    pickle.dump(mask_mat2, f)
with open('../../dataset/train_val_data_fine/mask_mat_full_sense','wb') as f:
    pickle.dump(mask_mat3, f)

In [21]:
hard_mask_mat1 = hard_make_mask_matrix(vocab_sense1)
hard_mask_mat2 = hard_make_mask_matrix(vocab_sense2)
hard_mask_mat3 = hard_make_mask_matrix(vocab_sense3)

with open('../../dataset/train_val_data_fine/hard_mask_mat_lex','wb') as f:
    pickle.dump(hard_mask_mat1, f)
with open('../../dataset/train_val_data_fine/hard_mask_mat_sense','wb') as f:
    pickle.dump(hard_mask_mat2, f)
with open('../../dataset/train_val_data_fine/hard_mask_mat_full_sense','wb') as f:
    pickle.dump(hard_mask_mat3, f)

In [9]:
dict_sense_count1 = dict(sense_count1)
dict_sense_count2 = dict(sense_count2)
dict_sense_count3 = dict(sense_count3)

In [10]:
print(len(sense_count1), len(sense_count2), len(sense_count3))

44 180 300


In [11]:
data_x = []
data_pos = []
data_label1 = []
data_label2 = []
data_label3 = []

for i in range(len(data)):
    if not all(np.array(data[i][2])==None) and (len(data[i][1])<=200):
        data_label1.append([ss if ss is not None and dict_sense_keys[ss][3] in dict_sense_count1 else None for ss in data[i][2]])
        data_label2.append([ss if ss is not None and dict_sense_keys[ss][4] in dict_sense_count2 else None for ss in data[i][2]])
        data_label3.append([ss if ss is not None and dict_sense_keys[ss][5] in dict_sense_count3 else None for ss in data[i][2]])
        data_x.append(data[i][1])
        data_pos.append(data[i][4])

In [12]:
def data_prepare(sense_id, x, pos, y, sense_count, lex_cond=False, pos_cond=False):
    num_examples = len(x)
    
    vocab_sense = [s for s, c in sense_count]
    vocab_sense = sorted(vocab_sense, key=lambda x:int(x[0]))
    sense2id = dict((s, i) for i,s in enumerate(vocab_sense))
    
    xx = np.zeros([num_examples, max_sent_size], dtype=int)
    xx_mask = np.zeros([num_examples, max_sent_size], dtype=bool)
    ss_mask = np.zeros([num_examples, max_sent_size], dtype=bool)
    yy = np.zeros([num_examples,max_sent_size], dtype=int)
    y_lex = np.zeros([num_examples, max_sent_size], dtype=int)
    y_pos = np.zeros([num_examples, max_sent_size], dtype=int)
    y_sense_pos = np.zeros([num_examples, max_sent_size], dtype=int)
    
    for j in range(num_examples):
        for i in range(max_sent_size):
            if(i>=len(x[j])):
                break
            w = x[j][i]
            s = y[j][i]
            p = pos[j][i]
            xx[j][i] = word2id[w] if w in word2id else word2id['UNKNOWN_TOKEN']
            xx_mask[j][i] = True
            ss_mask[j][i] = True if s is not None and dict_sense_keys[s][sense_id] in vocab_sense else False
            yy[j][i] = sense2id[dict_sense_keys[s][sense_id]] if s is not None and dict_sense_keys[s][sense_id] in vocab_sense else 0
            y_sense_pos[j][i] = int(dict_sense_keys[s][1]) - 1 if s is not None else 0
            if(lex_cond):
                y_lex[j][i] = lex2id[dict_sense_keys[s][3]] if s is not None and dict_sense_keys[s][3] in vocab_lex else len(vocab_lex)
            if(pos_cond):
                y_pos[j][i] = pos2id[p] if p in vocab_pos else len(vocab_pos)
        
    return xx, xx_mask, ss_mask, yy, y_lex, y_pos, y_sense_pos

In [13]:
data_x = np.array(data_x)
data_pos = np.array(data_pos)

def train_val_data(name, sense_id, index, split_label, data_label, sense_count, sampling_list, lex_cond=False, pos_cond=False, sampling=False):
    
    index_train, index_val, label_train_id, label_val_id = train_test_split(index, split_label, train_size=0.8, shuffle=True, stratify=split_label, random_state=0)
  
    if(sampling):
        dict_sample = dict(sampling_list)
        sm = RandomOverSampler(ratio=dict_sample)
        index_train1 = np.array(index_train).reshape(-1, 1)
        sampled_index, _ = sm.fit_sample(index_train1, label_train_id)
        count = Counter(_)
        count = count.most_common()
        sampled_index_train = np.array(sampled_index).reshape(1, -1)
        index_train = sampled_index_train[0]
    
    data_label = np.array(data_label)
    x_train = data_x[index_train]
    y_train = data_label[index_train]
    x_val = data_x[index_val]
    y_val = data_label[index_val]
    pos_train = []
    pos_val = []
    
    if(pos_cond):
        pos_train = data_pos[index_train]
        pos_val = data_pos[index_val]

    x_id_train, mask_train, sense_mask_train, y_id_train, lex_train, pos_id_train, sense_pos_id_train  = data_prepare(sense_id, x_train, pos_train, y_train, sense_count, lex_cond=lex_cond, pos_cond=pos_cond)
    x_id_val, mask_val, sense_mask_val, y_id_val, lex_val, pos_id_val, sense_pos_id_val = data_prepare(sense_id, x_val, pos_val, y_val, sense_count, lex_cond=lex_cond, pos_cond=pos_cond)

    train_data = {'x':x_id_train,'x_mask':mask_train, 'sense_mask':sense_mask_train, 'y':y_id_train, 'sense_pos':sense_pos_id_train, 'lex':lex_train, 'pos':pos_id_train}
    val_data = {'x':x_id_val,'x_mask':mask_val, 'sense_mask':sense_mask_val, 'y':y_id_val, 'sense_pos':sense_pos_id_val, 'lex':lex_val, 'pos':pos_id_val}
    
    with open('../../dataset/train_val_data_fine/all_word_'+ name,'wb') as f:
        pickle.dump([train_data,val_data], f)
    
    print(len(x_id_train)+len(x_id_val))

In [14]:
split_label1 = []
split_label2 = []
split_label3 = []

index1 = []
index2 = []
index3 = []

for jj, lab in enumerate(data_label1):
    min_idx = np.argmin([dict_sense_count1[dict_sense_keys[lab[i]][3]] if lab[i] is not None else np.inf for i in range(len(lab))  ]) 
    if(lab[min_idx] is not None):
        index1.append(jj)
        split_label1.append(dict_sense_keys[lab[min_idx]][3])

for jj, lab in enumerate(data_label2):
    min_idx = np.argmin([dict_sense_count2[dict_sense_keys[lab[i]][4]] if lab[i] is not None else np.inf for i in range(len(lab))  ]) 
    if(lab[min_idx] is not None):
        index2.append(jj)
        split_label2.append(dict_sense_keys[lab[min_idx]][4])

for jj, lab in enumerate(data_label3):
    min_idx = np.argmin([dict_sense_count3[dict_sense_keys[lab[i]][5]] if lab[i] is not None else np.inf for i in range(len(lab))  ]) 
    if(lab[min_idx] is not None):
        index3.append(jj)
        split_label3.append(dict_sense_keys[lab[min_idx]][5])
    
print(len(split_label1))
print(len(split_label2))
print(len(split_label3))

850083
838757
828921


In [15]:
train_val_data('lex', 3, index1, split_label1, data_label1, sense_count1, [], lex_cond=False, pos_cond=True)
train_val_data('sense', 4, index2, split_label2, data_label2, sense_count2, [], lex_cond=True, pos_cond=True)
train_val_data('full_sense', 5, index3, split_label3, data_label3, sense_count3, [], lex_cond=True, pos_cond=True)

/home/sshanukr/env/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


850083
838757
828921


In [16]:
sampled_sense_count1 = [('1:19', 10000),
 ('1:17', 10000),
 ('2:34', 10000),
 ('2:33', 10000),
 ('1:27', 10000),
 ('2:37', 8000),
 ('1:24', 8000),
 ('1:08', 8000),
 ('1:12', 7000),
 ('1:22', 5000),
 ('2:29', 5000),
 ('1:05', 3000),
 ('1:16', 3000),
 ('1:25', 3000),
 ('1:20', 3000),
 ('1:13', 2000)]

In [17]:
sampled_sense_count2= []
for s, c in sense_count2[120:]:
    sampled_sense_count2.append((s, 5000))
for s, c in sense_count2[75:120]:
    sampled_sense_count2.append((s, 8000))
for s, c in sense_count2[25:75]:
    sampled_sense_count2.append((s, 12000))

In [18]:
sampled_sense_count3= []
for s, c in sense_count3[130:]:
    sampled_sense_count3.append((s, 5000))
for s, c in sense_count3[70:130]:
    sampled_sense_count3.append((s, 8000))
for s, c in sense_count3[25:70]:
    sampled_sense_count3.append((s, 12000))

In [19]:
train_val_data('lex_sampled', 3, index1, split_label1, data_label1, sense_count1, sampled_sense_count1, lex_cond=False, pos_cond=True, sampling=True)
train_val_data('sense_sampled', 4, index2, split_label2, data_label2, sense_count2, sampled_sense_count2, lex_cond=True, pos_cond=True, sampling=True)
train_val_data('full_sense_sampled', 5, index3, split_label3, data_label3, sense_count3, sampled_sense_count3, lex_cond=True, pos_cond=True, sampling=True)

/home/sshanukr/env/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


909119
1814988
2375783
